# 시설 데이터 전처리 및 개수 파악

In [43]:
# 해당 코드를 실행하기 위한 패키지 설치 목록

!pip install pandas numpy folium geopandas scikit-learn shapely

In [44]:
import pandas as pd
import re

df = pd.read_csv('data/facilities.csv')

columns_to_drop = [
    'CTGRY_ONE_NM', 'CTGRY_TWO_NM', 'CTGRY_THREE_NM',
    'CTPRVN_NM', 'SIGNGU_NM',
    'RDNMADR_NM', 'LAST_UPDT_DE'
]
df_cleaned = df.drop(columns=columns_to_drop)

# 표준 시설 유형 정의
def standardize_facility(info):
    if pd.isna(info):
        return '기타'

    info = info.replace(" ", "")  # 공백 제거
    count = 0
    matched = []

    if '병원' in info:
        matched.append('동물병원')
        count += 1
    if '약국' in info:
        matched.append('동물약국')
        count += 1
    if '카페' in info or '호텔' in info or '용품' in info or '미용' in info:
        matched.append('반려동물편의시설')
        count += 1

    if count == 1:
        return matched[0]
    elif count > 1:
        return '보류'
    else:
        return '기타'

# 표준화 적용
df_cleaned['표준시설유형'] = df_cleaned['FCLTY_INFO_DC'].apply(standardize_facility)

# 결과 확인
print(df_cleaned[['FCLTY_INFO_DC', '표준시설유형']].head(20))

   FCLTY_INFO_DC    표준시설유형
0           동물약국      동물약국
1           애견카페  반려동물편의시설
2           동물약국      동물약국
3           동물약국      동물약국
4         일반동물병원      동물병원
5           동물약국      동물약국
6         일반동물병원      동물병원
7         일반동물병원      동물병원
8         일반동물병원      동물병원
9         일반동물병원      동물병원
10        일반동물병원      동물병원
11        일반동물병원      동물병원
12        일반동물병원      동물병원
13        일반동물병원      동물병원
14        일반동물병원      동물병원
15        일반동물병원      동물병원
16        일반동물병원      동물병원
17        일반동물병원      동물병원
18        일반동물병원      동물병원
19        일반동물병원      동물병원


In [45]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

df = df_cleaned

# 2. 위도(LA), 경도(LO) → Point 객체로 변환
geometry = [Point(xy) for xy in zip(df['LC_LO'], df['LC_LA'])]
gdf_facilities = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")  # WGS84

# 3. 서울시 행정동 경계 GeoJSON 불러오기
gdf_seoul = gpd.read_file("data/area.geojson")

# 4. 좌표계 통일 (GeoJSON도 EPSG:4326이라면 생략 가능)
gdf_seoul = gdf_seoul.to_crs("EPSG:4326")

# 5. 공간 조인: 시설이 어느 행정동 내에 있는지 매핑
gdf_joined = gpd.sjoin(gdf_facilities, gdf_seoul, how="inner", predicate="within")

# 6. 필요한 컬럼 선택 (GeoJSON의 컬럼명 확인 필요)
result = gdf_joined[[
    'FCLTY_NM',
    'LC_LA', 'LC_LO', 'geometry', 'dong', 'sggnm', '표준시설유형'
]]

# 동별 표준시설유형 개수 집계
facility_counts_by_gu = result.groupby(['dong', '표준시설유형']).size().unstack(fill_value=0).reset_index()

# 결과 출력
print(facility_counts_by_gu.head(20))

표준시설유형  dong  기타  동물병원  동물약국  반려동물편의시설  보류
0       가락1동   1     0     3         1   0
1       가락2동   1     2    10         2   0
2       가락본동   0     1    10         6   0
3       가리봉동   0     0     2         1   0
4        가산동   0     1    12         7   0
5       가양1동   4     0    14        10   0
6       가양2동   1     3     2         2   0
7       가양3동   0     0     2         0   0
8        가회동   8     0     0         0   0
9       갈현1동   0     3     3         2   0
10      갈현2동   0     2     9         6   0
11       강일동   0     2     5         1   0
12      개봉1동   0     3     3         2   0
13      개봉2동   0     2     6         2   0
14      개봉3동   0     1     4         1   0
15      개포2동   2     4     2         0   0
16      개포3동   0     0     2         0   0
17      개포4동   0     5     2         1   0
18      거여1동   0     3     0         2   0
19      거여2동   0     1     4         0   0


In [46]:
df_mapping = pd.read_csv("data/dong.csv", encoding="utf-8-sig")

# 1. 컬럼명 맞추기 위해 df_mapping 복사 및 컬럼명 통일
df_mapping_renamed = df_mapping.rename(columns={"행정동": "dong"})

# 2. 병합 (왼쪽 기준: mapping → 모든 행정동 유지)
merged = pd.merge(df_mapping_renamed, facility_counts_by_gu, on="dong", how="left")

# 3. 결측치 → 0으로 대체
merged.fillna(0, inplace=True)

# 4. 숫자형으로 변환 (결측치였던 값들이 float으로 들어올 수 있음)
for col in merged.columns[2:]:
    merged[col] = pd.to_numeric(merged[col], errors="coerce").fillna(0).astype(int)

# 공원 데이터 전처리

In [47]:
import geopandas as gpd
from shapely.geometry import Point

# 1. 서울시 공원현황 CSV 불러오기
df_parks = pd.read_csv("data/park.csv", encoding="cp949")

# 2. ‘소재지지번주소’가 있는 경우, 서울특별시만 필터링
if "소재지지번주소" in df_parks.columns:
    df_parks = df_parks[df_parks["소재지지번주소"]
                        .str.contains(r"^서울특별시", na=False)].reset_index(drop=True)

# 3. 컬럼명 통일 (실제 CSV 컬럼명에 맞추어 수정)
df_parks = df_parks.rename(columns={
    "공원명":       "녹지대명",
    "공원면적":         "녹지대면적",
})

# 4. 숫자형으로 변환 (문자열→NaN), 면적도 숫자로
df_parks["경도"]       = pd.to_numeric(df_parks["경도"], errors="coerce")
df_parks["위도"]       = pd.to_numeric(df_parks["위도"], errors="coerce")
df_parks["녹지대면적"] = pd.to_numeric(df_parks["녹지대면적"], errors="coerce")

# 5. 좌표·이름·면적 누락 행 제거
df_parks = (
    df_parks
    .dropna(subset=["경도", "위도", "녹지대명", "녹지대면적"])
    .reset_index(drop=True)
)

# 6. Point geometry 생성 → GeoDataFrame 선언 (CRS=WGS84)
geometry  = [Point(xy) for xy in zip(df_parks["경도"], df_parks["위도"])]
gdf_parks = gpd.GeoDataFrame(df_parks, geometry=geometry, crs="EPSG:4326")

# 7. 행정동 경계 GeoJSON 불러오기
gdf_seoul = gpd.read_file("data/area.geojson", encoding="utf-8")

# 8. CRS 통일
if gdf_parks.crs != gdf_seoul.crs:
    gdf_parks = gdf_parks.to_crs(gdf_seoul.crs)

# 9. 공간 조인: 공원이 속한 행정동 매핑
gdf_joined = gpd.sjoin(
    gdf_parks,
    gdf_seoul[["geometry", "dong"]],
    how="left",
    predicate="within"
)
gdf_joined = gdf_joined.drop(columns=["index_right"], errors="ignore")

# 10. DataFrame으로 변환 후 필요한 컬럼만 선택
df_result = pd.DataFrame(gdf_joined.drop(columns="geometry"))
selected  = df_result[["녹지대명", "녹지대면적", "dong"]]

# 11. ‘녹지대명’ 중복 제거 (첫 출현 유지)
selected = selected.drop_duplicates(subset=["녹지대명"], keep="first").reset_index(drop=True)

# 12. dong별로 공원명 쉼표 연결, 면적 합산
aggregated = selected.groupby("dong", as_index=False).agg({
    "녹지대명":   lambda names: ", ".join(names.astype(str)),
    "녹지대면적": "sum"
})

# 13. 결과 확인
print(aggregated.head())

   dong                                               녹지대명     녹지대면적
0  가락1동                                         가락1(시영아파트)   19159.2
1  가락2동                          개롱, 샛팽이, 안산골, 투구봉, 새말, 대건   29046.6
2  가락본동                웃말, 가락, 건너말, 팔각정, 방죽, 봉우리, 비석거리, 훙이   57683.5
3   가산동                                        조마, 무아래, 골말    3650.6
4  가양1동  푸르미, 마곡지구4호, 탑산(가양동), 놋다리, 똘고랑, 선두암, 한다리, 가로공원...  739115.9


In [48]:
# 2. 'dong' 컬럼을 기준으로 병합 (왼쪽 기준: selected_green)
combined = pd.merge(
    aggregated,
    merged,
    on="dong",
    how="outer"
)

# 3. 병합 후 누락된 값( NaN )을 0으로 채우기
combined.fillna(0, inplace=True)

# 4. 숫자형 컬럼들(int 타입)로 변환
#    'dong', '녹지대명', '구명' 같은 문자열 컬럼은 제외하고 변환합니다.
for col in combined.columns:
    if col not in ["dong", '구']: # 필요하면 "녹지대명"도 제외할 수 있음
        combined[col] = (
            pd.to_numeric(combined[col], errors="coerce")
              .fillna(0)
              .astype(int)
        )

combined.head(10)

,dong,녹지대명,녹지대면적,구,기타,동물병원,동물약국,반려동물편의시설,보류
0,가락1동,0,19159,송파구,1,0,3,1,0
1,가락2동,0,29046,송파구,1,2,10,2,0
2,가락본동,0,57683,송파구,0,1,10,6,0
3,가리봉동,0,0,구로구,0,0,2,1,0
4,가산동,0,3650,금천구,0,1,12,7,0
5,가양1동,0,739115,강서구,4,0,14,10,0
6,가양2동,0,32917,강서구,1,3,2,2,0
7,가양3동,0,23615,강서구,0,0,2,0,0
8,가회동,0,10004,종로구,8,0,0,0,0
9,갈현1동,0,4365,은평구,0,3,3,2,0


# 면적데이터 추가

In [49]:
area = pd.read_csv("data/area.csv", encoding="utf-8-sig")
area = area.drop(columns=["구"]) 
area["면적(m2)"] = area["면적"] * 1000000

# 2. ‘dong’을 기준으로 결합
merged_area = pd.merge(
    combined,
    area,
    on="dong",
    how="left"
)

# 3. 결측치(NA) → 0으로 채우기
merged_area.fillna(0, inplace=True)

merged_area.drop(columns=["녹지대명", "기타", "보류"], inplace=True)
# 5. 결과 확인 (상위 5개)
merged_area.head(5)

,dong,녹지대면적,구,동물병원,동물약국,반려동물편의시설,면적,면적(m2)
0,가락1동,19159,송파구,0,3,1,1.34,1340000.0
1,가락2동,29046,송파구,2,10,2,0.96,960000.0
2,가락본동,57683,송파구,1,10,6,1.13,1130000.0
3,가리봉동,0,구로구,0,2,1,0.40,400000.0
4,가산동,3650,금천구,1,12,7,2.52,2520000.0


# 반려동물 수 전처리

In [50]:
# 2. 구별 반려동물 수 파일 불러오기
merged_pet_gu = pd.read_csv("data/pet_count.csv", encoding="cp949")  
#   → 컬럼: ['구', '반려동물수']

# 3. 구별 총면적(m2) 계산 및 비율 컬럼 추가
merged_area["구_총면적(m2)"] = merged_area.groupby("구")["면적(m2)"].transform("sum")
merged_area["구면적(m2)대비비율"] = (merged_area["면적(m2)"] / merged_area["구_총면적(m2)"]) * 100
merged_area = merged_area.drop(columns=["구_총면적(m2)"])

# 4. 면적(m2) 대비 비율이 담긴 merged_area와 반려동물 수 merged_pet_gu를 '구' 기준으로 병합
merged_merged = pd.merge(
    merged_area,
    merged_pet_gu,
    on="구",
    how="left"
)

# 5. '구면적(m2)대비비율'을 활용해 동별 반려동물 수 비례 배분
merged_merged["동별_반려동물수"] = merged_merged["반려동물수"] * merged_merged["구면적(m2)대비비율"]

# 6. 동별 반려동물 수를 정수로 표현(원단위 절사/반올림 등 원하는 방식으로)
#    예: 소수점 버리고 정수로 만들려면 astype(int) 사용
merged_merged["동별_반려동물수"] = merged_merged["동별_반려동물수"].astype(int)

# 7. 필요한 컬럼만 남겨서 결과 확인
result = merged_merged[["구", "dong", "면적(m2)", "구면적(m2)대비비율","동별_반려동물수"]]
print(result.head())

     구  dong     면적(m2)  구면적(m2)대비비율  동별_반려동물수
0  송파구  가락1동  1340000.0     3.955136    150314
1  송파구  가락2동   960000.0     2.833530    107688
2  송파구  가락본동  1130000.0     3.335301    126758
3  구로구  가리봉동   400000.0     1.988072     43671
4  금천구   가산동  2520000.0    19.354839    283045


# 인구 데이터 추가

In [51]:

# 1. 원본 CSV 불러오기 (실제 경로·인코딩에 맞게 수정)
pop = pd.read_csv("data/pop.csv", encoding="cp949")

# 2. 컬럼명 변경
pop = pop.rename(columns={
    "행정구역": "dong",
    "2025년05월_총인구수": "인구수",
    "2025년05월_세대수": "세대수"
})

pop.drop(columns=["2025년05월_세대당 인구", "2025년05월_남자 인구수", "2025년05월_여자 인구수", "2025년05월_남여 비율"], inplace=True)

# 3. “dong”에 “서울” 포함된 행만 남기기
pop = pop[pop["dong"].str.contains("서울")].reset_index(drop=True)

# 4. 괄호가 있는 행(dong 문자열에 '(' 포함) 제거
pop["dong"] = pop["dong"].str.replace(r"\s*\(.*?\)", "", regex=True).str.strip()
# 'dong' 컬럼에서 '제'가 숫자+동 앞에만 붙어 있을 때만 제거하기
pop["dong"] = pop["dong"].str.replace(r"제(?=\d+동)", "", regex=True)

# 5. “서울특별시 ○○구 ○○동” 형식에서 시·구·동을 추출
#    - 정규표현식으로 정확히 3개 그룹(시, 구, 동)만 뽑아냅니다.
#    - 예: "서울특별시 종로구 사직동" → 시="서울특별시", 구="종로구", 동="사직동"
pattern = r"^(서울특별시)\s+([^ ]+구)\s+(.+동)$"
extracted = pop["dong"].str.extract(pattern)

# 5-1. 추출 결과 컬럼명이 0,1,2 → 각각 “시”, “구”, “dong”으로 바꿔서 데이터프레임에 붙이기
extracted.columns = ["시", "구", "dong_small"]
pop = pd.concat([pop, extracted], axis=1)

# 5-2. 정규표현식에 매칭되지 않은(=NaN이 생긴) 행들은 제거
pop = pop[pop["dong_small"].notna()].reset_index(drop=True)

# 6. 원래의 'dong' 컬럼은 더 이상 필요 없으니 제거한 뒤, 'dong_small'을 'dong'으로 이름 변경
pop = pop.drop(columns=["dong", "시", "구"])
pop = pop.rename(columns={"dong_small": "dong"})

for col in ["인구수", "세대수"]:
    pop[col] = (
        pop[col]
        .astype(str)
        .str.replace(",", "", regex=False)
    )
    pop[col] = pd.to_numeric(pop[col], errors="coerce").fillna(0).astype(int)

# 'dong' 컬럼에 있는 모든 마침표(.)를 중간점(·)으로 교체하기
pop["dong"] = pop["dong"].str.replace(".", "·", regex=False)

merged_pop = pd.merge(
    merged_merged,
    pop,
    on="dong",
    how="left"
)

merged_pop.head(10)

# 7. 최종 확인
merged_pop.head(10)


,dong,녹지대면적,구,동물병원,동물약국,반려동물편의시설,면적,면적(m2),구면적(m2)대비비율,반려동물수,동별_반려동물수,인구수,세대수
0,가락1동,19159,송파구,0,3,1,1.34,1340000.0,3.955136,38005,150314,26974.0,9650.0
1,가락2동,29046,송파구,2,10,2,0.96,960000.0,2.833530,38005,107688,30506.0,12369.0
2,가락본동,57683,송파구,1,10,6,1.13,1130000.0,3.335301,38005,126758,24570.0,11576.0
3,가리봉동,0,구로구,0,2,1,0.40,400000.0,1.988072,21967,43671,8724.0,6051.0
4,가산동,3650,금천구,1,12,7,2.52,2520000.0,19.354839,14624,283045,25148.0,18607.0
5,가양1동,739115,강서구,0,14,10,4.70,4700000.0,11.341699,37800,428716,33548.0,21055.0
6,가양2동,32917,강서구,3,2,2,1.00,1000000.0,2.413127,37800,91216,13064.0,7905.0
7,가양3동,23615,강서구,0,2,0,0.50,500000.0,1.206564,37800,45608,13571.0,7591.0
8,가회동,10004,종로구,0,0,0,0.54,540000.0,2.258469,10698,24161,3711.0,1851.0
9,갈현1동,4365,은평구,3,3,2,0.97,970000.0,3.265993,31725,103613,14129.0,7211.0


# 펫프렌들리 지수 산출(PCA 방식)

In [52]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler

#  2-1. 시설 총합 (동물병원 + 동물약국 + 반려동물편의시설)
merged_pop["시설총합"] = merged_pop["동물병원"] + merged_pop["동물약국"] + merged_pop["반려동물편의시설"]

#  2-2. 반려동물 1마리당 시설 수 (작을수록 부족하므로, 뒤에서 정규화 시 부호를 반대로 볼 수 있음)
merged_pop["시설_대_펫비율"] = merged_pop["시설총합"] / merged_pop["동별_반려동물수"]

#  2-3. 반려동물 1마리당 녹지대 면적 (㎡ 단위)
merged_pop["녹지_대_펫비율"] = merged_pop["녹지대면적"] / merged_pop["동별_반려동물수"]

#  2-4. 인구 1명당 반려동물 비율 (높을수록 반려동물 보급률이 높음)
merged_pop["펫_대_인구비율"] = merged_pop["동별_반려동물수"] / merged_pop["인구수"]

#  2-5. 반려동물 1마리당 동 면적 (㎡ 단위)
merged_pop["면적_대_펫비율"] = merged_pop["면적(m2)"] / merged_pop["동별_반려동물수"]

#  2-6. 세대당 반려동물 비율 (높을수록 반려동물을 키우는 가구 비중이 높음)
merged_pop["펫_대_세대비율"] = merged_pop["동별_반려동물수"] / merged_pop["세대수"]

# 3. 각 지표를 Min-Max 정규화 (0~1 사이)
#    정규화 함수 정의
def min_max_norm(series):
    return (series - series.min()) / (series.max() - series.min())

merged_pop["시설총합_norm"] = min_max_norm(merged_pop["시설총합"])

# 3-1. 시설_대_펫비율 (펫당 시설 → 높을수록 좋으니 그대로)  
merged_pop["시설_대_펫비율_norm"] = min_max_norm(merged_pop["시설_대_펫비율"])

# 3-2. 녹지_대_펫비율 (높을수록 좋음)
merged_pop["녹지_대_펫비율_norm"] = min_max_norm(merged_pop["녹지_대_펫비율"])

# 3-4. 면적_대_펫비율 (높을수록 좋음)
merged_pop["면적_대_펫비율_norm"] = min_max_norm(merged_pop["면적_대_펫비율"])


# 1) PCA 입력용 데이터 준비
features = [
    "시설총합_norm",
    "시설_대_펫비율_norm",
    "녹지_대_펫비율_norm",
    "면적_대_펫비율_norm"
]
df_pca = merged_pop.dropna(subset=features)[features]

# 2) 표준화 (평균0, 분산1)
scaler = StandardScaler()
X_std = scaler.fit_transform(df_pca)

# 3) PCA(n_components=3) 적용하여 PC1, PC2, PC3 추출
pca = PCA(n_components=3, random_state=42)
pcs = pca.fit_transform(X_std)    # shape = (n_samples, 3)
evr = pca.explained_variance_ratio_

print("▶ 주성분별 설명분산비율:", np.round(evr, 4))

# 4) loadings 방향 보정 (각 PC마다 음수가 더 많으면 뒤집기)
loadings = pca.components_        # shape = (3, n_features)
for i in range(3):
    if (loadings[i] < 0).sum() > (loadings[i] > 0).sum():
        pcs[:, i] = -pcs[:, i]

# 5) 각 PC를 0~1로 Min–Max 스케일링
mins = pcs.min(axis=0)
maxs = pcs.max(axis=0)
pcs_norm = (pcs - mins) / (maxs - mins)

# 6) merged_pop에 PC1~PC3 컬럼 추가
for i in range(3):
    merged_pop.loc[df_pca.index, f"PC{i+1}"] = pcs_norm[:, i]

# 7) (선택) PC1~3을 분산비율 가중합하여 단일지수로 결합
#    weight_sum = evr.sum()  → 1.0이 될 것이므로 곧바로 evr를 가중치로 사용
merged_pop.loc[df_pca.index, "펫프렌들리지수"] = (
      evr[0] * pcs_norm[:, 0]
    + evr[1] * pcs_norm[:, 1]
    + evr[2] * pcs_norm[:, 2]
)

# 8) 결과 확인
print(merged_pop[["dong", "PC1", "PC2", "PC3", "펫프렌들리지수"]].head())


▶ 주성분별 설명분산비율: [0.4103 0.2859 0.2224]
   dong       PC1       PC2       PC3   펫프렌들리지수
0  가락1동  0.080049  0.109661  0.183237  0.104947
1  가락2동  0.313927  0.091614  0.214165  0.202634
2  가락본동  0.353973  0.087385  0.226899  0.220689
3  가리봉동  0.116870  0.117601  0.174034  0.120279
4   가산동  0.322280  0.062869  0.225511  0.200367


# 분위수를 활용한 창업 추천 로직

In [53]:
# 1) 부족항목 계산을 위한 기준값(녹지·시설 비율 하위 20%) 구하기
green_thresh_norm    = merged_pop["녹지_대_펫비율_norm"].quantile(0.2)
facility_thresh_norm = merged_pop["시설_대_펫비율_norm"].quantile(0.2)

# 2) check_need 함수 수정
def check_need(row):
    need = []
    if row["녹지_대_펫비율_norm"] < green_thresh_norm:
        need.append("공원 부족")
    if row["시설_대_펫비율_norm"] < facility_thresh_norm:
        need.append("시설 부족")
    return "-" if not need else ", ".join(need)

merged_pop["부족항목"] = merged_pop.apply(check_need, axis=1)

# 3) 서울시 행정동 경계 GeoJSON 불러오기
admin_gdf = gpd.read_file("data/area.geojson", encoding="utf-8")

# 4) 'merged_pop'과 행정동 경계를 'dong' 기준으로 병합
merged_gdf = admin_gdf.merge(merged_pop, on="dong", how="left")

# 5) '인접동' 컬럼을 위해 빈 값을 미리 할당
merged_gdf["인접동"] = None

# 6) GeoPandas의 .touches()로 경계가 맞닿은(인접한) 동들을 찾아 '인접동'에 저장
for idx, row in merged_gdf.iterrows():
    neighbors = merged_gdf[
        (merged_gdf.geometry.touches(row.geometry))  # 경계만 닿으면 True
        & (merged_gdf.index != idx)                  # 자기 자신 제외
    ]["dong"].tolist()
    merged_gdf.at[idx, "인접동"] = neighbors

# 7) 동별 시설 수만 별도 DataFrame으로 준비 (인덱스는 'dong')
facilities_df = merged_pop[["dong", "동물병원", "동물약국", "반려동물편의시설"]].set_index("dong")

# 8) “시설 부족”인 동에 대해 인접 동 대비 어떤 시설이 부족한지 추천
def recommend_resources(row):
    recs = []

    # 1) 시설 부족 시 로직
    if "시설 부족" in row["부족항목"]:
        neighbors = row["인접동"]
        if neighbors:
            neigh_counts = facilities_df.loc[neighbors]
            avg_hospital  = neigh_counts["동물병원"].mean()
            avg_pharmacy  = neigh_counts["동물약국"].mean()
            avg_facility  = neigh_counts["반려동물편의시설"].mean()

            my_hospital  = row["동물병원"]
            my_pharmacy  = row["동물약국"]
            my_facility  = row["반려동물편의시설"]

            fac_recs = []
            if my_hospital < avg_hospital * 0.5:
                fac_recs.append("동물병원")
            if my_pharmacy < avg_pharmacy * 0.5:
                fac_recs.append("동물약국")
            if my_facility < avg_facility * 0.5:
                fac_recs.append("편의시설")

            if fac_recs:
                recs.append(f"{', '.join(fac_recs)} 창업 추천")
            else:
                recs.append("인접동에 시설 충분 → 추가 추천 없음")
        else:
            recs.append("인접동 정보 없음 → 시설 추천 불가")

    # 2) 공원 부족 시, 별도 추천 문구 추가
    if "공원 부족" in row["부족항목"]:
        recs.append("공원 공간 확보·녹지 조성 검토 추천")

    # 3) 아무 추천도 없으면 '-' 반환
    return "-" if not recs else ", ".join(recs)

# 적용
merged_gdf["추천시설"] = merged_gdf.apply(recommend_resources, axis=1)

# 10) 결과 확인 (dong, 부족항목, 인접동, 추천시설)
print(merged_gdf[["dong", "부족항목", "인접동", "추천시설"]].head(10))

          dong   부족항목                                                인접동  \
0          사직동      -            [무악동, 교남동, 종로1·2·3·4가동, 청운효자동, 소공동, 명동]   
1          삼청동  시설 부족           [부암동, 평창동, 가회동, 종로1·2·3·4가동, 청운효자동, 성북동]   
2          부암동  공원 부족                [삼청동, 평창동, 청운효자동, 홍제3동, 홍제2동, 홍은1동]   
3          평창동  시설 부족  [삼청동, 부암동, 정릉3동, 정릉4동, 성북동, 녹번동, 불광1동, 진관동, 홍은1동]   
4          무악동      -                 [사직동, 교남동, 청운효자동, 천연동, 홍제1동, 홍제2동]   
5          교남동      -                          [사직동, 무악동, 소공동, 천연동, 충현동]   
6          가회동  시설 부족                       [삼청동, 종로1·2·3·4가동, 혜화동, 성북동]   
7  종로1·2·3·4가동      -  [사직동, 삼청동, 가회동, 종로5·6가동, 이화동, 청운효자동, 혜화동, 명동, ...   
8      종로5·6가동      -     [종로1·2·3·4가동, 이화동, 창신1동, 창신2동, 광희동, 을지로동, 신당동]   
9          이화동      -             [종로1·2·3·4가동, 종로5·6가동, 창신2동, 혜화동, 삼선동]   

                     추천시설  
0                       -  
1        동물병원, 동물약국 창업 추천  
2    공원 공간 확보·녹지 조성 검토 추천  
3        동물병원, 편의시설 창업 추천  
4                      

# Folium 활용 시각화

In [54]:
import folium

# 1) folium 지도 생성
m = folium.Map(location=[37.5665, 126.9780], zoom_start=11)

# 3) Choropleth 지도 추가
geo_data = merged_gdf.to_json()
pet_index_df = merged_gdf[["dong", "펫프렌들리지수"]]

values = pet_index_df["펫프렌들리지수"]
quantiles = list(values.quantile([0, 0.25, 0.5, 0.9, 1.0]))
bins = quantiles

folium.Choropleth(
    geo_data=geo_data,
    name="펫 지수(5단계)",
    data=pet_index_df,
    columns=["dong", "펫프렌들리지수"],
    key_on="feature.properties.dong",
    fill_color="Blues",
    fill_opacity=0.7,
    line_opacity=0.2,
    bins=bins,  # 5단계로 나누기
    legend_name="펫 프렌들리 지수",
    reset=True,
    highlight=True
).add_to(m)

# 4) GeoJsonTooltip 설정
folium.GeoJson(
    data=geo_data,
    name="정보 툴팁",
    style_function=lambda feat: {
        "fillColor": "transparent",
        "color": "#000000",
        "weight": 0.5,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["dong", "펫프렌들리지수", "부족항목", "추천시설"],
        aliases=["행정동", "펫 프렌들리 지수", "부족 항목", "추천시설"],
        localize=True,
        sticky=True,
        labels=True,
        style=(
            "background-color: white; "
            "color: #333333; "
            "font-family: Arial; "
            "font-size: 12px; "
            "padding: 5px;"
        ),
        parse_html=False
    )
).add_to(m)

# 5) 레이어 컨트롤
folium.LayerControl().add_to(m)

# 6) 결과 출력
m